# Workflow to calculate the quantum efficiency of a polycrystalline material

In [ ]:
import subprocess
import nglview as nv

from ase.calculators.castep import Castep
from ase.atoms import Atoms
from ase.io import read, write
from ase.visualize import view
from ase.build import surface
import ase.calculators.castep
import ase.io.castep
from ase.io.castep import read_castep_cell

from pymatgen.ext.matproj import MPRester
from pymatgen.core import Lattice, Structure, Molecule
from pymatgen.core.surface import SlabGenerator, generate_all_slabs, Slab
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.analysis.wulff import WulffShape
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.electronic_structure.bandstructure import BandStructure
from pymatgen.symmetry.kpath import KPathLatimerMunro
from pymatgen.vis.structure_chemview import quick_view

from wulffpack import SingleCrystal

from functions import *

I recommend using pymatgen to generate the bulk/slab and then convert it to an ASE atom object. The advantages of this approach are:
- pymatgen has a better bulk/slab generation algorithm 
- castep can link a calculator to the Atom object


#### Download and save .cif file for necessary materials

In [ ]:
import json
api_key = ''
with MPRester(api_key) as m:
    results = m.get_dos_by_material_id("mp-30")
#print(type(results))
#res_json = results.as_dict()
with open('./structures/jsons/DOS/Cu_bulk_dos_ref.json', 'w') as f:
    json.dump(results.as_dict(), f)

#### Generate the bulk structure and bulk input

In [ ]:
bulk = Structure.from_file(filename="Cu_metal_fcc.cif")
#view(AseAtomsAdaptor().get_atoms(bulk))
bulk_seed = 'Cu_bulk_vic_thesis'
castep_opt_template = {
    'directory': f"./structures/{bulk_seed}",
    'label': bulk_seed,
    # Param File Instructions
    'task': 'Spectral', #Choose: SinglePoint, BandStructure, Spectral
    'spectral_task' : 'ALL', #Choose if task is Spectral: DOS, BandStructure, Optics, CoreLoss, All 
    'xc_functional': 'PBE',
    'energy_cutoff': 600,
    'elec_energy_tol': 1e-8,
    'opt_strategy': 'Speed',
    'fix_occup' : False,
    'mixing_scheme' : 'Broydon', #Choose from Broydon or Pulay
    'smearing_width' : 300,
    'spin_polarized': False,
    'max_scf_cycles': 1000,
    'write_potential': False,
    'write_density': False,
    'extra_bands': True,
    #Cell File Instructions
    'kpoints': (32,32,1),
    'snap_to_symmetry': False,
    'fix_all_cell': False,
    'continuation': False,
    'bandstruct_path': 'GXWKGLUWLK',
    'bandstruct_kpt_dist': 0.0184,
    'spectral_kpt_grid': (10,10,10),
    'calculate_pdos': True 
}
PBS_options = {
    'seed_name': bulk_seed,
    'tasks_seeds': [['singlepoint', bulk_seed], ['Optados', bulk_seed]], #Choose one or multiple (carefully!): SinglePoint, BandStructure, Spectral, OptaDOS
    'queue': 'debug',
    'castep_version': 'castep_19' #choose from castep_19, castep_18, castep_18_mod
}
OptaDOS_options = {
    'seed_name': bulk_seed,
    'optados_task': 'pdos', # Choose: dos(default), compare_dos, compare_jdos, jdos, pdos, optics, core, all
    'broadening': 'adaptive', #Choose: adaptive(default), fixed, linear
    'iprint': '1', #Choose: 1 - bare minimum, 2 - with progress reports, 3 - fulld debug output
    'efermi': 'optados', #Choose: optados - recalculate, file - read from CASTEP file, insulator - count filled bands, float - supplied by user
    'dos_spacing': '0.001', #DOS spacing in unit (default: eV): default - 0.1
    'pdos': 'angular' #Choose: angular, species_ang, species, sites or more detailed descriptions such as: 
    #PDOS : sum:Si1-2(s) - sum of s-chnnls on 2 Si atms (1 proj), 
    #PDOS : Si1;Si2(s) - DOS on Si atom 1 and DOS on s-channel of Si atom 2 (2 proj) 
}
OptaDOS_photoemission_opt= {
    'work_function' : 4.556,
    's_area' : 6.339744873,
    'slab_volume' : 190.942338,
    'elec_field' : 0,
    'imfp_const' : 19.0,
    'JDOS_SPACING' : 0.1,
    'JDOS_MAX_ENERGY' : 25,
    'BROADENING' : 'linear',
    'OPTICS_GEOM' : 'unpolar',
    'optics_qdir' : [1, 1.000, 1.0000],
    'photon_energy' : 21.2,
    'linear_smearing' : 0.026,
    'fixed_smearing' :  0.026,
    'optics_intraband' : True,
    'photo_model' : '1step',
    'momentum' : 'crystal',
    'hybrid_linear' : True,
    'temp' : 300,
    'theta_lower' : 59,
    'theta_upper' : 61,
    'photo_output' : 'be'
}
generate_castep_input(calc_struct=AseAtomsAdaptor().get_atoms(bulk), **castep_opt_template)
generate_qsub_file(**PBS_options)
generate_optados_input(OptaDOS_options, **OptaDOS_photoemission_opt)

#### Generate the surface structure

In [ ]:
bulk = Structure.from_file(filename="Cu_metal_fcc.cif")
bulk_ase = AseAtomsAdaptor().get_atoms(bulk)
print(bulk_ase)
print(bulk)
surface = ase.build.surface(lattice = bulk_ase, indices = (1,1,1), layers = 5, vacuum=15, tol=1e-10, periodic=True)

In [ ]:

def create_slab_layer_convergence(structure, indices, min, max, seed, *cutoff):
    layers = [i for i in range(min, max+1)]
    castep_opt_template = {
    'directory': f"./structures/{seed}",
    'label': seed,
    # Param File Instructions
    'task': 'Singlepoint', #Choose: SinglePoint, BandStructure, Spectral
    'spectral_task' : 'DOS', #Choose if task is Spectral: DOS, BandStructure, Optics, CoreLoss, All 
    'xc_functional': 'PBE',
    'energy_cutoff': 600,
    'elec_energy_tol': 1e-8,
    'opt_strategy': 'Speed',
    'fix_occup' : False,
    'mixing_scheme' : 'Broydon',
    'smearing_width' : 300,
    'spin_polarized': False,
    'max_scf_cycles': 1000,
    'write_potential': False,
    'write_density': False,
    'extra_bands': True,
    #Cell File Instructions
    'kpoints': (16,16,16),
    'snap_to_symmetry': False,
    'fix_all_cell': False,
    'continuation': False,
    'bandstruct_path': 'GXWKGLUWLK',
    'bandstruct_kpt_dist': 0.0184,
    'spectral_kpt_grid': (10,10,10),
    'calculate_pdos': True 
}
    PBS_options = {
        'seed_name': seed,
        'tasks_seeds': [], #Choose one or multiple (carefully!): SinglePoint, BandStructure, Spectral, OptaDOS
        'queue': 'debug',
        'castep_version': 'castep_19' #choose from castep_19, castep_18, castep_18_mod
    }
    for i in layers:
        surface = ase.build.surface(lattice = structure, indices =indices, layers = i, vacuum=15, tol=1e-10, periodic=True)
        temp_opt = castep_opt_template
        temp_opt['directory'] = f"./structures/{seed}"
        temp_opt['label'] = f"{seed}_{i}L"
        temp_opt['kpoints'] = get_adjusted_kpts(structure, surface, [9,9,9])
        PBS_options['tasks_seeds'].append(['singlepoint',temp_opt['label']])
        if cutoff != None:
            temp_opt['cutoff'] = cutoff
        generate_castep_input(calc_struct=surface, **temp_opt)
    generate_qsub_file(**PBS_options)
    return;

In [ ]:
surf_vic = read_cell2pmg('./structures/Cu.cell')
surf_vic
view(AseAtomsAdaptor.get_atoms(surf_vic))
surfaces = {
    'Cu_surf_111' : ase.build.surface(lattice = bulk_ase, indices = (1,1,1), layers = 16, vacuum=15, tol=1e-10, periodic=True),
    'Cu_surf_100' : ase.build.surface(lattice = bulk_ase, indices = (1,0,0), layers = 16, vacuum=15, tol=1e-10, periodic=True),
    'Cu_surf_110' : ase.build.surface(lattice = bulk_ase, indices = (1,1,0), layers = 16, vacuum=15, tol=1e-10, periodic=True)
}
seed = 'Cu_surf_111'
castep_opt_template = {
    'directory': f"./structures/{seed}",
    'label': seed,
    # Param File Instructions
    'task': 'Spectral', #Choose: SinglePoint, BandStructure, Spectral
    'spectral_task' : 'ALL', #Choose if task is Spectral: DOS, BandStructure, Optics, CoreLoss, All 
    'calculate_pdos': True,
    'xc_functional': 'PBEsol',
    'energy_cutoff': 600,
    'elec_energy_tol': 1e-8,
    'opt_strategy': 'Speed',
    'fix_occup' : False,
    'mixing_scheme' : 'Pulay', #Choose from Broydon or Pulay
    'smearing_width' : 300,
    'spin_polarized': False,
    'max_scf_cycles': 1000,
    'write_potential': True,
    'write_density': True,
    'extra_bands': True,
    #Cell File Instructions
    'kpoints': (32,32,1),
    'snap_to_symmetry': False,
    'generate_symmetry': True,
    'fix_all_cell': True,
    'continuation': False,
    'bandstruct_path': 'GXWKGLUWLK',
    'bandstruct_kpt_dist': 0.0184,
    'spectral_kpt_grid': (32,32,1)    
}
PBS_options = {
    'seed_name': seed,
    'tasks_seeds': [['Spectral', seed], ['Optados', seed]], #Choose one or multiple (carefully!): SinglePoint, BandStructure, Spectral, OptaDOS
    'queue': 'debug',
    'castep_version': 'castep_19' #choose from castep_19, castep_18, castep_18_mod
}
OptaDOS_options = {
    'seed_name': seed,
    'optados_task': 'pdos', # Choose: dos(default), compare_dos, compare_jdos, jdos, pdos, optics, core, all, photoemission
    'broadening': 'adaptive', #Choose: adaptive(default), fixed, linear
    'iprint': '1', #Choose: 1 - bare minimum, 2 - with progress reports, 3 - fulld debug output
    'efermi': 'optados', #Choose: optados - recalculate, file - read from CASTEP file, insulator - count filled bands, float - supplied by user
    'dos_spacing': '0.001', #DOS spacing in unit (default: eV): default - 0.1
    'pdos': 'angular' #Choose: angular, species_ang, species, sites or more detailed descriptions such as: 
    #PDOS : sum:Si1-2(s) - sum of s-chnnls on 2 Si atms (1 proj), 
    #PDOS : Si1;Si2(s) - DOS on Si atom 1 and DOS on s-channel of Si atom 2 (2 proj) 
}
OptaDOS_photoemission_opt= {
    'work_function' : 4.556,
    'surface_area' : 6.339744873,
    'slab_volume' : 190.942338,
    'elec_field' : 0,
    'imfp_const' : 19.0,
    'JDOS_SPACING' : 0.1,
    'JDOS_MAX_ENERGY' : 25,
    'BROADENING' : 'linear',
    'OPTICS_GEOM' : 'unpolar',
    'optics_qdir' : [1, 1, 1],
    'photon_energy' : 21.2,
    'linear_smearing' : 0.026,
    'fixed_smearing' :  0.026,
    'optics_intraband' : True,
    'photo_output' : 'be',
    'photo_model' : '1step',
    'momentum' : 'crystal',
    'hybrid_linear' : True,
    'temp' : 300,
    'theta_lower' : 59,
    'theta_upper' : 61
}

generate_castep_input(calc_struct=surfaces[seed], **castep_opt_template)
generate_optados_input(OptaDOS_options, **OptaDOS_photoemission_opt)
generate_qsub_file(**PBS_options)

#### Run the calculations

In [ ]:
#folder_change_command = 'cd {}'.format(seed)
#submit_command = 'qsub {}.qsub'.format(seed)

#subprocess.call(folder_change_command)
#subprocess.call(submit_command)
#subprocess.call('cd ..')

#### Extract the final energies

In [ ]:
bulk_tot_energy, bulk_fermi = get_energies(seed=bulk_seed)
surf_111_tot_energy, surf_111_fermi = get_energies(seed=surf_seed)
#print(bulk_tot_energy, surf_111_tot_energy)

#### Calculate the surface energy

In [ ]:
surf_energy_111 = calc_surface_energy(bulk_tot_energy, surf_111_tot_energy,len(surface_111.numbers),surface_111.cell.volume / surface_111.cell[2][2])
factor = 16.02176565
print('The surface energy of the {} surface is: '.format(surf_seed) + str(surf_energy_111) + ' eV/Angstrom^2.\n' + 'This is equal to: ' + str(surf_energy_111*factor) + ' J/m^2')


#### Get the % of each surface from a  Wulff construction

In [ ]:
mat_structure = bulk
# facets_energies = {
#     (1,0,0) : 0.1161,
#     (1,1,0) : 0.1211,
#     (1,1,1) : 0.1055
# } #energies in eV/Angstrom^2
facets_energies = {
    (1,0,0) : 1.86,
    (1,1,0) : 1.94,
    (1,1,1) : 1.69
} #energies in J/m^2

different_fractions = get_wulff_fractions(AseAtomsAdaptor.get_atoms(bulk),facets_energies)

# Testing of performance by pymatgen wulff shape class
# Results: the ratios of the facets is different than expected and seems incorrect with the wulffshape class

# facets = [(1,0,0),(1,1,0),(1,1,1)]
# energies = [1.86,1.94,1.69]
# def new_wulff_fractions(mat_structure, indices,facet_energies):
#     lattice = SpacegroupAnalyzer(mat_structure).find_primitive()
#     print(lattice)
#     shape = WulffShape(lattice.lattice, indices, facet_energies)
#     return shape.area_fraction_dict
# facets_fractions =  get_wulff_fractions(AseAtomsAdaptor.get_atoms(bulk),facets_energies)
# different_fractions = get_wulff_fractions(AseAtomsAdaptor.get_atoms(bulk),facets_energies)
# print(facets_fractions)
# print(different_fractions)

#### Launch OptaDOS

#### Get QE from OptaDOS output

#### Calculate the weighted average QE

#### Calculate and display plenty more properties (DOS, bands, ...)